In [1]:
import os
import pandas as pd

In [2]:
def get_stat(seqFilepath, key):
    """
    given fastqFilepath, search stat file and return value correspond to key
    """
    
    statFilepath=seqFilepath.split('.fastq')[0] + ".stat"
    if not(os.path.isfile(statFilepath)):
        print("ERROR: stat file not exists, {}".format(statFilepath))
        return None
    stat_df=pd.read_csv(statFilepath, delimiter='\t')
    if not(key in stat_df.columns):
        print("ERROR: key not exists, {}".format(key))
        return None
    return stat_df.loc[0, key]

In [3]:
metaFilepath="mizuho_metadata_dna.csv"
meta_df=pd.read_csv(metaFilepath)
meta_df.head()

,fastq_id,sample_id,direction,filepath,filepath_google,filepath_sqc,MFC_id,elapsed_months,position,position_sub,read_length,size_bp
0,MFC2_03m_anode_1_R1,MFC2_03m_anode_1,R1,/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_03...,/work/GoryaninU/mitsuki/mizuho/dna/row/3months...,/work/sqc_data/GoryaninU/150217_M02137_0066_00...,2,3,anode,NaN,310,1112588447
1,MFC2_03m_anode_1_R2,MFC2_03m_anode_1,R2,/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_03...,/work/GoryaninU/mitsuki/mizuho/dna/row/3months...,/work/sqc_data/GoryaninU/150217_M02137_0066_00...,2,3,anode,NaN,310,1391003532
2,MFC2_03m_anode_2_R1,MFC2_03m_anode_2,R1,/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_03...,/work/GoryaninU/mitsuki/mizuho/dna/row/3months...,/work/sqc_data/GoryaninU/140627_M02137_0045_00...,2,3,anode,NaN,310,363318360
3,MFC2_03m_anode_2_R2,MFC2_03m_anode_2,R2,/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_03...,/work/GoryaninU/mitsuki/mizuho/dna/row/3months...,/work/sqc_data/GoryaninU/140627_M02137_0045_00...,2,3,anode,NaN,310,423863194
4,MFC2_03m_chamber2_1_R1,MFC2_03m_chamber2_1,R1,/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_03...,/work/GoryaninU/mitsuki/mizuho/dna/row/3months...,/work/sqc_data/GoryaninU/140627_M02137_0045_00...,2,3,chamber,chamber2,310,415939242


In [4]:
dct_lst=[]
baseDirec="/work/GoryaninU/mitsuki/mizuho/dna"
for fastqId in meta_df["fastq_id"]:
    step_lst=["row", "trim", "correct", "filter"]
    fp_lst=[]
    fp_lst.append("{}/row/{}.fastq.gz".format(baseDirec, fastqId))
    fp_lst.append("{}/trim/{}.fastq".format(baseDirec, fastqId))
    fp_lst.append("{}/filter/{}.tmp.fastq".format(baseDirec, fastqId))
    fp_lst.append("{}/filter/{}.fastq".format(baseDirec, fastqId))
    assert len(step_lst)==len(fp_lst)
    
    dct={"fastq_id": fastqId}
    for step, fp in zip(step_lst, fp_lst):
        dct[step]=get_stat(fp, "num_seqs")
    dct_lst.append(dct)

In [5]:
out_df=pd.DataFrame(dct_lst)
out_df=out_df[["fastq_id", "row", "trim", "correct", "filter"]]
out_df["per"]=out_df["filter"]/out_df["row"]*100
out_df.head()

,fastq_id,row,trim,correct,filter,per
0,MFC2_03m_anode_1_R1,4830140,4823635,3504562,3292101,68.157465
1,MFC2_03m_anode_1_R2,4830140,4823635,3683251,3292101,68.157465
2,MFC2_03m_anode_2_R1,1459909,1457722,823750,736816,50.469995
3,MFC2_03m_anode_2_R2,1459909,1457722,886691,736816,50.469995
4,MFC2_03m_chamber2_1_R1,1744919,1734885,885377,784211,44.942545


In [6]:
out_df.to_csv("./out/num_seqs.csv", index=False)

In [ ]:
seqFilepath="/work/GoryaninU/mitsuki/mizuho/dna/row/MFC2_36m_channel8_R2.fastq.gz"
get_num_seqs(seqFilepath)